In [50]:
# import libraries
import osmnx as ox
import random
import pandas as pd
import folium
import bisect
import networkx as nx
import requests
import pickle
import csv

from geopy.distance import geodesic
from networkx.algorithms.shortest_paths.weighted import _weight_function
from heapq import heappop, heappush
from itertools import count
from functools import lru_cache
from geopy.geocoders import Nominatim
from functools import reduce

In [2]:
# finding the coordinates of a place from the address
def get_coordinates(location_name):
    # Initialize the Nominatim geocoder
    geolocator = Nominatim(user_agent="Modified_A_start_Search_in_OSMNX")
    location = geolocator.geocode(location_name)
    if location:
        latitude = location.latitude
        longitude = location.longitude
        return longitude, latitude
    else:
        print("Unable to find the coordinates of the location")


In [3]:
# Load the graph of place
# specify the location
place_name = "Delhi, India"
#G = ox.graph_from_place(place_name, network_type='drive', simplify=True)
# pickle.dump(G, open('filename.pickle', 'wb'))
# load the delhi graph
G = pickle.load(open("delhi.pickle", "rb"))
charging_stations = ox.geometries_from_place(place_name, tags={'amenity': 'charging_station'})

C:\Users\91735\AppData\Local\Temp\ipykernel_9840\3490614042.py:8: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  charging_stations = ox.geometries_from_place(place_name, tags={'amenity': 'charging_station'})


In [53]:
'''charging_distances = {}
charging_nodes = {}

for i, station in charging_stations.iterrows():
    if station['geometry'].geom_type != 'Point':
        continue
    x, y = station['geometry'].x, station['geometry'].y
    nearest_node, dist = ox.nearest_nodes(G, x, y, return_dist=True)
    charging_distances[nearest_node] = dist
    charging_nodes[nearest_node] = (x, y)

charging_stations = []
columns = ["longitude", "lattitude", "nearest_node", "distance"]
for node in charging_distances:
    charging_stations.append([charging_nodes[node][0], charging_nodes[node][1], node, charging_distances[node]])
with open("charging_stations.csv", "w") as f:
    writter = csv.writer(f)
    writter.writerow(columns)
    writter.writerows(charging_stations)'''
print()

In [63]:
# get the charging info
charging_stations = pd.read_csv("charging_stations.csv")

In [65]:
charging_nodes = {}
charging_distances = {}
# iterate through the dataframe
for i, station in charging_stations.iterrows():
    # get the coordinates
    x = station["longitude"]
    y = station["lattitude"]
    # get the nearest node
    nearest_node = station["nearest_node"]
    # get the distance
    dist = station["distance"]
    charging_nodes[nearest_node] = (x, y)
    charging_distances[nearest_node] = dist

In [5]:
def get_edge_distance(u, v, weight='length'):
    weight = _weight_function(G, weight)
    return weight(u, v, G.adj[u][v])

In [89]:
def astar_path(G, source, target, charging_distances, weight='length', max_range=15, charge=100):
    """Returns a list of nodes in a shortest path between source and target using the modified A* algorithm.

    There may be more than one shortest path.  This returns only one.

    Parameters
    ----------
    G : NetworkX graph

    source : node
        Starting node for path

    target : node
        Ending node for path
    
    charging_destinations : dict
        A dictionary containing the nodes that are closest to the charging stations
        & their distances from from the charging stations
    
    weight : string
        The edge data key corresponding to the edge weight
    
    max_range : int
        The maximum distance that the vehicle can travel on a single charge in kms
    
    charge : int
        The initial charge of the vehicle in percentage

    Raises NetworkXNoPath
        If no path exists between source and target.
    """

    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    # define function to find distance from target node
    @lru_cache(maxsize=None)
    def distance_from_target(u, v):
        return geodesic((G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])).meters
    
    push = heappush
    pop = heappop
    weight = _weight_function(G, weight)

    G_succ = G._adj # For speed-up (and works for both directed and undirected graphs)

    # The queue stores priority, node, cost to reach, and parent.
    # Uses Python heapq to keep in priority order.
    # Add a counter to the queue to prevent the underlying heap from
    # attempting to compare the nodes themselves. The hash breaks ties in the
    # priority and is guaranteed unique for all nodes in the graph.

    c = count()
    queue = [(0, charge, next(c), source, 0, None)] # 0 cost to reach source

    # Maps enqueued nodes to distance of discovered paths and the
    # computed heuristics to target. We avoid computing the heuristics
    # more than once and inserting the node into the queue too many times.
    enqueued = {}
    # Maps explored nodes to parent closest to the source.
    explored = {}

    while queue:
        # pop the smallest item from queue
        _, curcharge, _, curnode, dist, parent = pop(queue)

        # check if the node has a charging station nearby
        if curnode in charging_distances:
            curcharge = 100

        if curnode == target:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            return path, curcharge
        
        if curnode in explored:
            # Do not override the parent of starting node
            if explored[curnode] is None:
                continue
            
            if enqueued[curnode] < dist:
                continue
        
        explored[curnode] = parent

        for neighbor, w in G_succ[curnode].items():
            edge_weight = weight(curnode, neighbor, w)
            ncost = dist + edge_weight
            new_charge = curcharge - edge_weight/(max_range*10) # old charge - percentage battery used to travel edge_weight
            # If the new charge is less than 0, the vehicle cannot reach the neighbor
            if new_charge < 0:
                continue
 
            if neighbor in enqueued:
                # A less costly path is already known
                if enqueued[curnode] <= ncost:
                    continue
            h = distance_from_target(neighbor, target)

            enqueued[neighbor] = ncost
            push(queue, ((ncost + h) * (400 - new_charge)/400, new_charge, next(c), neighbor, ncost, curnode))
    
    return list(enqueued.keys()), None


In [7]:
# get source coordinates
# source_coordinates = get_coordinates("source address")
source_coordinates = (77.0851248, 28.555524) # for Delhi International Airport

In [8]:
# get destination coordinates
# destination_coordinates = get_coordinates('destionation address')
destination_coordinates = (77.24079592327321, 28.65608095) # for Red Fort


In [139]:

source_node, distance_from_source = ox.nearest_nodes(G, *source_coordinates, return_dist=True)
destination_node, distance_from_destination = ox.nearest_nodes(G, *destination_coordinates, return_dist=True)

# find the path from source node to destination node
path, charge = astar_path(G=G, source=source_node, target=destination_node, charging_distances=charging_distances, weight='length', max_range=13.4, charge=100)


In [140]:

if not charge:
    print("No path found")
else:
    distance = sum(get_edge_distance(path[i], path[i+1]) for i in range(len(path)-1))
    print(f"Distance: {distance:,.2f} meters")

Distance: 29,511.50 meters


In [141]:
# Create a Folium map centered around location
curmap = folium.Map(location=[28.5499202, 77.1142483], zoom_start=12)  # center coordinates

# Add the shortest path to the map as a polyline
# Get coordinates of nodes in the shortest path
path_coords = [(source_coordinates[1], source_coordinates[0])]
for node in path:
    path_coords.append((G.nodes[node]['y'], G.nodes[node]['x']))
    # add the path to the charging station & back to the node
    if node in charging_nodes:
        path_coords.append((charging_nodes[node][1], charging_nodes[node][0]))
        path_coords.append((G.nodes[node]['y'], G.nodes[node]['x']))
#path_coords += [(destination_coordinates[1], destination_coordinates[0])]

folium.PolyLine(locations=path_coords, color='blue', weight=5, opacity=0.7).add_to(curmap)
folium.Marker(location=(source_coordinates[1], source_coordinates[0]), icon=folium.Icon(color='blue')).add_to(curmap)
folium.Marker(location=(destination_coordinates[1], destination_coordinates[0]), icon=folium.Icon(color='blue')).add_to(curmap)
# add charging station points in the graph
"""for node in G.nodes:
    if random.choice(range(100)) != 5:
        continue
    coordinates = G.nodes[node]['y'], G.nodes[node]['x']
    folium.Marker(location=coordinates, icon=folium.Icon(color='green')).add_to(curmap)"""
'''for i, station in charging_stations.iterrows():
    if station['geometry'].geom_type == 'Point':
        folium.Marker(location=[station['geometry'].y, station['geometry'].x], icon=folium.Icon(color='red')).add_to(curmap)
'''
for node in charging_distances:
    #if node in shortest_path_nodes:
    folium.Marker(location=[G.nodes[node]['y'], G.nodes[node]['x']], icon=folium.Icon(color='green')).add_to(curmap)

# Save the map to an HTML file or display it
curmap.save('map.html')  # Save as HTML file
curmap  # Display the map in Jupyter Notebook or IPython

In [75]:
import folium
import time
from IPython.display import display

# Create a Folium map centered around location
curmap = folium.Map(location=[28.5499202, 77.1142483], zoom_start=12)  # center coordinates

# Function to add a marker to the map with a delay
def add_marker_with_delay(location, color, delay):
    time.sleep(delay)
    folium.Marker(location=location, icon=folium.Icon(color=color)).add_to(curmap)

# Add the markers with a delay
for coords in path_coords:
    add_marker_with_delay(coords, 'blue', 0.5)  # Add blue marker for each node

# Display the updated map
display(curmap)
